In [61]:
from qdrive.dataset import generate_dataset_info

# import a converter
from data_converter.converter_txt_to_hdf5 import TxtToHdf5Converter


# Create dataset info with sample_name/timestamp/ structure
# Single T1 decay dataset for a randomly selected sample
from datetime import datetime, timedelta
import os
import json
import numpy as np
import random

data_path = "data/folder_base_data/"

# Randomly select one sample
sample_names = ["sample_A", "sample_B", "sample_C"]
sample_name = random.choice(sample_names)

# Create single T1 decay measurement
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
path = f"{data_path}/{sample_name}/{timestamp}/"
os.makedirs(path, exist_ok=True)

# Create T1 measurement data
t1_times = np.linspace(0, 100, 50)  # microseconds
t1_signal = np.exp(-t1_times/25) + 0.02 * np.random.randn(50)  # T1 = 25 μs
np.savetxt(f"{path}measurement.txt", np.column_stack([t1_times, t1_signal]), 
           header="Time(us)\tSignal", delimiter="\t")

# Metadata JSON for T1 decay measurement
metadata = {
    "measurement_type": "T1_decay",
    "qubit_id": "Q1",
    "temperature": 0.015,  # Kelvin
    "power": -20,  # dBm
    "frequency": 5.2e9,  # Hz
    "fitted_T1": 25.3,  # microseconds
    "measurement_date": datetime.now().isoformat()
}
with open(f"{path}metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

generate_dataset_info(path,
    dataset_name = "T1_decay_measurement",
    creation = datetime.now(),
    description="T1 decay measurement for qubit relaxation time characterization.",
    attributes={"sample" : sample_name, "qubit_id": "Q1", "measurement_type": "T1_decay"},
    keywords=["T1", "decay", "relaxation", "characterization"],
    # converters = [TxtToHdf5Converter],
    # skip=["*.json", "raw_data/*"]
)

In [60]:
from etiket_client.sync.backends.sources import add_sync_source
from etiket_client.python_api.scopes import get_scope_by_name, get_selected_scope

from etiket_client.sync.backends.filebase.filebase_sync_class import FileBaseSync
from etiket_client.sync.backends.filebase.filebase_config_class import FileBaseConfigData
import qdrive

from pathlib import Path
data_path = Path("./data/folder_base_data/")

if not data_path.exists():
    raise ValueError(f"Data path {data_path} does not exist. Please correct the path.")

# scope to which the data will be uploaded
scope4upload = get_selected_scope()

# sample name and set up will be added to every dataset that is uploaded from this location
config = FileBaseConfigData(root_directory=data_path, server_folder = False)

# give a name to the sync agent (should be locally unique)
add_sync_source('my_sync_source_name', FileBaseSync, config, scope4upload)

qdrive.restart_sync_agent()
